In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json

In [0]:
# get the output parameters from the previous notebook
dbutils.widgets.text("bronze_path", "")
dbutils.widgets.text("silver_path", "")
dbutils.widgets.text("gold_path", "")

bronze_path= dbutils.widgets.get("bronze_path")
silver_path= dbutils.widgets.get("silver_path")
gold_path= dbutils.widgets.get("gold_path")

In [0]:
# # comment later while running with the pipeline
# # ADLS paths
# tiers = ["bronze", "silver", "gold"]
# adls_paths = {tier: f"abfss://{tier}@adlschinook.dfs.core.windows.net" for tier in tiers}
# bronze_path = adls_paths["bronze"]
# silver_path = adls_paths["silver"]
# gold_path = adls_paths["gold"]
# bronze_path

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [0]:
# list the files inside the bronze firectory
silver_directory= dbutils.fs.ls(f"{silver_path}/dbo")
# split the filename to just get the tablenames
table_names =[]
for files in silver_directory:
    table_names.append(files.name.split("/")[0])
table_names

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [0]:
# iterate through each file and make transformations if necessary and write ti the gold ADLS
for name in table_names:
    file_path = f"{silver_path}/dbo/{name}"
    df = spark.read\
        .format("parquet")\
            .load(file_path)

    if name == "Customer" or name == "Employee":
        df= df.withColumn("Phone", regexp_replace("Phone", r"[^0-9 ]+", ""))\
            .withColumn("Fax", regexp_replace("Fax", r"[^0-9 ]+", ""))
        df= df.withColumn("CountryCode", split(col("Phone"), " ", 2).getItem(0))\
            .withColumn("Phone", split(col("Phone"), " ", 2).getItem(1))\
                .withColumn("Fax", split(col("Fax"), " ", 2).getItem(1))
        df= df.withColumn("Phone", regexp_replace("Phone", r"[^0-9]+", ""))\
            .withColumn("Fax", regexp_replace("Fax", r"[^0-9]+", ""))

    if name == "Invoice":
        df= df.withColumnRenamed("Total", "TotalPrice")
    
    try:
        df.write\
            .format("delta")\
                .option("overwriteSchema", True)\
                    .mode("overwrite")\
                        .save(f"{gold_path}/dbo/{name}")
        print(f"Success writing table {name} to the gold ADLS ")
    except Exception as e:
        print(f"Error writing table {name} to the gold ADLS")


      

Success writing table Album to the gold ADLS 
Success writing table Artist to the gold ADLS 
Success writing table Customer to the gold ADLS 
Success writing table Employee to the gold ADLS 
Success writing table Genre to the gold ADLS 
Success writing table Invoice to the gold ADLS 
Success writing table InvoiceLine to the gold ADLS 
Success writing table MediaType to the gold ADLS 
Success writing table Playlist to the gold ADLS 
Success writing table PlaylistTrack to the gold ADLS 
Success writing table Track to the gold ADLS 
